Check what what percentage of the fractional statistical error is made up of boost statistical error for different boost fractional errors (to see if boost statistical error is negligible compared to shape noise).

For the shape-measurements methods method.

In [1]:
%matplotlib inline
import numpy as np
import scipy.integrate 
import scipy.interpolate
import matplotlib.pyplot as plt
import constrain_IA_shapes_FudgeFactor as sh

SURVEY = 'LSST_DESI'

if SURVEY=='SDSS':
    import params as pa
elif SURVEY=='LSST_DESI':
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet"

In [2]:
# Get the boost
boost_fid = sh.get_boost(sh.rp_cents, pa.boost_assoc)
print "boost fid=", boost_fid

boost fid= [ 6.92854945  3.98930454  2.50727285  1.76        1.38320865  1.1932222
  1.09742687]


In [3]:
# Get the shape noise for both samples
shear_cov_1 = sh.shapenoise_cov(pa.e_rms_mean, sh.z_close_low, sh.z_close_high, boost_fid, sh.rp_cents, sh.rp_bins, pa.dNdzpar_fid, pa.pzpar_fid) 
shear_cov_2 = sh.shapenoise_cov(pa.e_rms_mean, sh.z_close_low, sh.z_close_high, boost_fid, sh.rp_cents, sh.rp_bins, pa.dNdzpar_fid, pa.pzpar_fid)

print "shear cov=", shear_cov_1

shear cov= [  1.45880562e-06   4.57400723e-07   1.31385285e-07   3.37901365e-08
   7.76192077e-09   1.62438835e-09   3.18852547e-10]


In [ ]:
shear_covar = [0]*len(pa.cov_perc)
for i in range(0,len(pa.cov_perc)):
    shear_covar[i] = sh.get_cov_btw_methods(shear_cov_1, shear_cov_2, pa.cov_perc[i])

In [ ]:
# Get Ncorr
corr_fac_fid = sh.N_corr(sh.rp_cents, pa.dNdzpar_fid, pa.pzpar_fid, pa.dNdzpar_fid, pa.pzpar_fid, boost_fid) 
print "corr_fac_fid=", corr_fac_fid

In [ ]:
# Set the boost statistical error as a fractional error on (B-1), since this is what I think we would measure. 
# Abs error on boost = abs error on boost - 1
boost_err = 0.1 * (boost_fid-1.)

# Compute the statistical error on Ncorr due to boost
sumW_insamp = sh.sum_weights(sh.z_close_low, sh.z_close_high, pa.zmin, pa.zmax, sh.z_close_low, sh.z_close_high, pa.zmin_ph, pa.zmax_ph, pa.e_rms_mean, sh.rp_cents, pa.dNdzpar_fid, pa.pzpar_fid)
sumW_intotal = sh.sum_weights(pa.zmin, pa.zmax, pa.zmin, pa.zmax, sh.z_close_low, sh.z_close_high, pa.zmin_ph, pa.zmax_ph, pa.e_rms_mean, sh.rp_cents, pa.dNdzpar_fid, pa.pzpar_fid)

sig_Ncorr = (boost_err / boost_fid**2) * np.sqrt(1. - np.asarray(sumW_insamp)/ np.asarray(sumW_intotal))

print "sigNcorr=", sig_Ncorr

In [ ]:
# Get fiducial gamma IA
gIA_fid= sh.gamma_fid(sh.rp_cents)
print "gIAfid=", gIA_fid

In [ ]:
# And get the percentage of the total statistical error that is boost statistical error, for a variety of choices of cov_perc and a

cov_mat_stat_tot = np.diag(np.zeros(len(shear_cov_1)))
cov_mat_stat_boost = np.diag(np.zeros(len(shear_cov_1)))
for ai in range(0,len(pa.a_con)):
    for ci in range(0,len(pa.cov_perc)):
        for ri in range(0,len(shear_cov_1)):
            cov_mat_stat_tot[ri,ri] = (1.-pa.a_con[ai])**2 * gIA_fid[ri]**2 * (( sig_Ncorr[ri]**2 / corr_fac_fid[ri]**2)  + (sh.subtract_var(shear_cov_1[ri], shear_cov_2[ri], shear_covar[ci][ri]) / (corr_fac_fid[ri]**2 * (1.-pa.a_con[ai])**2 * gIA_fid[ri]**2)))
            cov_mat_stat_boost[ri,ri] = (1.-pa.a_con[ai])**2 * gIA_fid[ri]**2 * ( sig_Ncorr[ri]**2 / corr_fac_fid[ri]**2)
        frac = np.diag(cov_mat_stat_boost) / np.diag(cov_mat_stat_tot)
        print "a=",pa.a_con[ai], "%=", pa.cov_perc[ci], "percentage=", frac*100